#### We write our notes and questions here in Google Docs [https://tinyurl.com/llamas-a2](https://tinyurl.com/llamas-a2) instead of a markdown file so it can be updated in real time.

# A2: Vector Semantics

Nikolai Ilinykh, Mehdi Ghanimifard, Wafia Adouane and Simon Dobnik

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read [the following instructions](https://github.com/sdobnik/computational-semantics/blob/master/README.md) on how to work on group assignments.

Write all your answers and the code in the appropriate boxes below.

In this lab we will look at how to build distributional semantic models from corpora and use semantic similarity captured by these models to do semantic tasks. We are also going to examine how different vector composition functions for phrases affect both the model and the learned information about similarities.  

Note that this lab uses a code from `dist_erk.py`, which contains functions that highly resemble those shown during the lecture. In the end, you can use either of the functions (from the lecture / from the file) to solve the tasks.

In [1]:
# the following command simply imports all the methods from that code.
from dist_erk import *

## 1. Loading a corpus

To train a distributional model, we first need a sufficiently large collection of texts which contain different words used frequently enough in different contexts. Here we will use a section of the Wikipedia corpus which you can download from [here](https://linux.dobnik.net/cloud/index.php/s/isMBj49jt5renYt?path=%2Fresources%2Fa2-distributional-representations) (wikipedia.txt.zip). (This file has been borrowed from another lab by [Richard Johansson](http://www.cse.chalmers.se/~richajo/)).  
When unpacked, the file is 151mb, hence if you are using the MLT servers you should store it in a temporary folder outside your home and adjust the `corpus_dir` path below.  
<!-- <It may already exist in `/opt/mlt/courses/cl2015/a5`.> -->


In [2]:
corpus_dir = 'wiki'

## 2. Building a model

Now you are ready to build the model.  
Using the methods from the code imported above build three word matrices with 1000 dimensions as follows:  

(i) with raw counts (saved to a variable `space_1k`);  
(ii) with PPMI (`ppmispace_1k`);  
(iii) with reduced dimensions SVD (`svdspace_1k`).  
For the latter use `svddim=5`. **[5 marks]**

Your task is to replace `...` with function calls. Functions are imported from `dist_erk.py` earlier, and they largely resemble functions shown during the lecture.

In [3]:
numdims = 1000
svddim = 5

# which words to use as targets and context words? >> filter stop words?
# we need to count the words and keep only the N most frequent ones >> top_vocab
# which function would you use here with which variable? 
ktw = do_word_count( corpus_dir, numdims)  # Top-1000 word from corpus

wi = make_word_index(ktw) # word index dict, where an item = word:idx
words_in_order = sorted(wi.keys(), key=lambda w:wi[w]) # [w1,w2...] in the order of wi-idx

# create different spaces (the original matrix space, the ppmi space, the svd space)   
# which functions with which arguments would you use here?
print('create count matrices')
space_1k = make_space(corpus_dir, wi, numdims)
print('ppmi transform') # *Positive Pointwise Mutual Information (PPMI): ppmi(w1,w2) = max( 0, log( p(w1,w2)/p(w1)*p(2) ) )
ppmispace_1k = ppmi_transform(space_1k, wi)
print('svd transform')
svdspace_1k = svd_transform(ppmispace_1k, numdims, svddim)
print('done.')

reading file wikipedia.txt
create count matrices
reading file wikipedia.txt
ppmi transform
svd transform
done.


In [4]:
# now, to test the space, you can print vector representation for some words

a_word='house'
space_dict = svdspace_1k # space_1k, ppmispace_1k, svdspace_1k
print(f'{a_word}:', space_dict[a_word])

house: [-4827.71077501   413.56039891  2667.80891551    64.61482227
    18.07211429]


Oxford Advanced Dictionary has 185,000 words, hence 1,000 words is not representative. We trained a model with 10,000 words, and 50 dimensions on truncated SVD. It took 40 minutes on a laptop. We saved all three matrices [here](https://linux.dobnik.net/cloud/index.php/s/isMBj49jt5renYt?path=%2Fresources%2Fa2-distributional-representations) (pretrained.zip). Download them and unpack them to a `pretrained` folder which should be a subfolder of the folder with this notebook:

In [5]:
import numpy as np

numdims = 10000
svddim = 50

print('Please wait...')
ktw_10k       = np.load('./pretrained/ktw_wikipediaktw.npy', allow_pickle=True)
space_10k     = np.load('./pretrained/raw_wikipediaktw.npy', allow_pickle=True).all()
ppmispace_10k = np.load('./pretrained/ppmi_wikipediaktw.npy', allow_pickle=True).all()
svdspace_10k  = np.load('./pretrained/svd50_wikipedia10k.npy', allow_pickle=True).all()
print('Done.')


Please wait...
Done.


In [6]:
# testing semantic space
print('house:', space_10k['house'])

house: [2554 3774 3105 ...    0    0    0]


In [177]:
import pandas as pd
# Matrix shapre of the space: 10000 words * vector numdim
pd.DataFrame.from_dict(space_10k ).transpose() # space_10k , ppmispace_10k, svdspace_10k 

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
,984656,394184,253602,333673,151135,109011,89281,239618,120815,115263,...,70,76,67,72,88,64,66,74,79,148
penguin,208,81,22,15,6,14,3,13,26,5,...,0,0,0,0,0,0,0,0,0,0
permit,43,63,5,20,2,48,111,3,6,3,...,0,0,0,0,0,0,0,0,0,0
45,10703,59,93,99,43,30,9471,136,228,199,...,0,0,0,0,0,0,0,0,0,0
felix,251,67,38,38,22,8,20,25,38,6,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
blair,254,36,32,44,42,8,18,14,14,11,...,0,0,0,0,0,0,0,0,0,0
tibetan,150,85,104,44,50,15,22,22,24,13,...,0,0,0,0,0,0,0,0,0,0
·,102,0,4,7,8,12,1,58,46,1,...,0,0,0,0,0,0,0,0,0,0
flown,35,54,5,19,26,13,53,7,8,17,...,0,0,0,0,0,0,0,0,0,0


## 3. Testing semantic similarity

The file `similarity_judgements.txt` (a copy is included with this notebook) contains 7,576 pairs of words and their lexical and visual similarities (based on the pictures) collected through crowd-sourcing using Mechanical Turk as described in [1]. The score range from 1 (highly dissimilar) to 5 (highly similar). Note: this is a different dataset from the phrase similarity dataset we discussed during the lecture (the one from [2]). For more information, please read the papers.

The following code will transform similarity scores into a Python-friendly format:

In [29]:
word_pairs = [] # test suit word pairs
semantic_similarity = [] 
visual_similarity = []
test_vocab = set()  # unique words from pairs


for index, line in enumerate(open('similarity_judgements.txt')):
    data = line.strip().split('\t')  # format:  w1#w2 \t SemSim \t VisSim => ['w1#w2', 'SemSim', 'VisSim']
    if index > 0 and len(data) == 3:
        w1, w2 = tuple(data[0].split('#'))
        # it will check if both words from each pair exist in the word matrix.
        if w1 in ktw_10k and w2 in ktw_10k:
            word_pairs.append((w1, w2))
            test_vocab.update([w1, w2])
            semantic_similarity.append(float(data[1]))
            visual_similarity.append(float(data[2]))
        
# testvocab - {words in testvocab but not in ktw} == intersection of testvocab & ktw => words that are both in wordpairs and ktw
# print('number of available words to test:', len(test_vocab-(test_vocab-set(ktw)))) 
print('number of available words to test:', len(test_vocab&set(ktw_10k))) # intersect of testvocab & ktw10k => words both in wordpairs and ktw10k
# Pairs where both words are in top-10000-words
print('number of available word pairs to test:', len(word_pairs))


number of available words to test: 155
number of available word pairs to test: 774


In [25]:
list(zip(word_pairs, visual_similarity, semantic_similarity))[:10]  # wordpair, human-judged-sem-sim, human-judged-vis-sim 

[(('stick', 'sword'), 3.4, 2.6),
 (('cabin', 'cabinet'), 1.75, 2.25),
 (('chicken', 'sparrow'), 3.0, 4.5),
 (('bag', 'gate'), 1.0, 1.5),
 (('bull', 'trumpet'), 1.0, 1.25),
 (('balloon', 'brick'), 1.0, 1.0),
 (('helicopter', 'jet'), 2.6, 4.6),
 (('cape', 'spider'), 1.0, 1.2),
 (('boots', 'mouse'), 1.0, 1.0),
 (('kite', 'marble'), 1.0, 1.0)]

Now we are going to test how the cosine similarity between vectors of each of the three spaces (normal space, ppmi, svd) compares with the human similarity judgements for the words in the similarity dataset. Which of the three spaces best approximates human judgements?

For comparison of several scores, we can use [Spearman correlation coefficient](https://en.wikipedia.org/wiki/Spearman's_rank_correlation_coefficient) which is implemented in `scipy.stats.spearmanr` [here](https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.stats.spearmanr.html). The values of the Sperman correlation coefficient range from -1, 0 to 1, where 0 indicates no correlation, 1 perfect correaltion and -1 negative correlation. Hence, the greater the number the better the similarity scores align. The p values tells us if the coefficient is statistically significant. For this to be the case, it must be less than or equal to $< 0.05$.

Here is how you can calculate Pearson's correlation coefficient betweeen the scores of visual similarity and semantic similarity of the available words in the test suite:

In [30]:
from scipy import stats

rho, pval = stats.spearmanr(semantic_similarity, visual_similarity) # args = 2 lists of values
print("""Visual Similarity vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))


Visual Similarity vs. Semantic Similarity:
rho     = 0.7122
p-value = 0.0000


Let's now calculate the cosine similarity scores of all word pairs in an ordered list using all three matrices. **[6 marks]**

In [32]:
raw_similarities  = [cosine(w1,w2, space_10k) for w1, w2 in word_pairs]
ppmi_similarities = [cosine(w1,w2, ppmispace_10k) for w1, w2 in word_pairs]
svd_similarities  = [cosine(w1,w2, svdspace_10k) for w1, w2 in word_pairs]

Now, calculate correlation coefficients between lists of similarity scores and the real semantic similarity scores from the experiment. The scores of what model best correlates them? Is this expected? **[6 marks]**

In [162]:
# your code should go here

def spearman_compare(sims1, sims2):
    rho, pval = stats.spearmanr(sims1, sims2) # args = 2 lists of values
    print("""
    rho     = {:.4f}
    p-value = {:.4f}\n""".format(rho, pval))

for name, sims in {'raw':raw_similarities, 'ppmi':ppmi_similarities, 'svd':svd_similarities}.items():
    print(f'{name} vs sematic similarities:')
    spearman_compare(semantic_similarity, sims)


raw vs sematic similarities:

    rho     = 0.1522
    p-value = 0.0000

ppmi vs sematic similarities:

    rho     = 0.4547
    p-value = 0.0000

svd vs sematic similarities:

    rho     = 0.4232
    p-value = 0.0000



**Your answer should go here:**

The vector similarities from PPMI and SVD spaces align much better with the human-judged sematic similarities than those from raw-count space. PPMI vectors similarity also perform slightly better.

We can also calculate correlation coefficients between lists of cosine similarity scores and the real visual similarity scores from the experiment. Which similarity model best correlates with them? How do the correlation coefficients compare with those from the previous comparison - and can you speculate why do we get such results? **[7 marks]**

In [164]:
# Your code should go here...
for name, sims in {'raw':raw_similarities, 'ppmi':ppmi_similarities, 'svd':svd_similarities}.items():
    print(f'{name} vs visual similarities:')
    spearman_compare(visual_similarity, sims)

raw vs visual similarities:

    rho     = 0.1212
    p-value = 0.0007

ppmi vs visual similarities:

    rho     = 0.3838
    p-value = 0.0000

svd vs visual similarities:

    rho     = 0.3097
    p-value = 0.0000



**Your answer should go here:**

The approximation of vector similarities vs human-judged visual also shows better results for PPMI and SVD, although lower than when comparing to semantic similarities. the p-value for raw space vs visual similarities is above 0 (but only slightly so negligible). A possible explanation is that the human-judged visial similarities were determined not solely on textual/linguistic contexts, but also within other modal contexts (eg vision), which are not captured by the distributional representation that only considers textual contexts.

## 4. Operations on similarities

We can perform mathematical operations on vectors to derive meaning predictions. For example, we can subtract the normalised vectors for `king` minus `queen` and add the resulting vector to `man` and we hope to get the vector for `woman`. Why? **[3 marks]**

**Your answer should go here:**

![title](./embeddings.png =250x)
Since the vector of each word embeds its semantic meaning in the given context, we assume that the distance and direction between king-queen will be close to that between man-woman in the same vector space. 

So assuming king -queen ≈ man -woman:

=> woman ≈ man -(king -queen) ie, man -manliness

=> woman ≈ man + (queen -king), ie, man +womanliness



Here is some helpful code that allows us to calculate such comparisons.

In [77]:
from scipy.spatial import distance

def normalize(vec):
    return vec / veclen(vec)

def find_similar_to(vec1, space):
    # vector similarity funciton (a,b) => 1-distance(a,b)
#     sim_fn = lambda a, b: 1-distance.euclidean(normalize(a), normalize(b))
#     sim_fn = lambda a, b: 1-distance.correlation(a, b)
    #sim_fn = lambda a, b: 1-distance.cityblock(normalize(a), normalize(b))
#     sim_fn = lambda a, b: 1-distance.chebyshev(normalize(a), normalize(b))
    #sim_fn = lambda a, b: np.dot(normalize(a), normalize(b))
    sim_fn = lambda a, b: 1-distance.cosine(a, b)

    sims = [ (word2, sim_fn(vec1, space[word2])) for word2 in space.keys() ] # each tuple = w2, 1-distance(vec1,vec2) 

    return sorted(sims, key = lambda p:p[1], reverse=True)  # sort by tuple[1] descendingly, ie sort by 1-distance

Here is how you apply this code. Comment on the results you get. **[3 marks]**

In [87]:
for name, space in {'raw':space_10k, 'ppmi':ppmispace_10k, 'svd':svdspace_10k}.items():
    short = normalize(space['short'])
    light = normalize(space['light'])
    long = normalize(space['long'])
    heavy = normalize(space['heavy'])

    sims = find_similar_to(light - (heavy - long), space)[:10]
    print(name,'\n', sims)

raw 
 [('long', 0.936140411029304), ('just', 0.9049060246775732), ('normal', 0.897070493693546), ('child', 0.8968235510083787), ('universal', 0.8954682113733906), ('genius', 0.8941840695017294), ('unit', 0.8939911177435964), ('bird', 0.893943608600412), ('light', 0.8930841816879834), ('radical', 0.8928522776983006)]
ppmi 
 [('long', 0.6455903190370874), ('light', 0.5756309144130762), ('short', 0.22665051391680424), ('about', 0.2062232575609404), ('around', 0.20341064658972596), ('than', 0.1997623801104953), ('longer', 0.19775518996857122), ('through', 0.19530678790677225), ('each', 0.19315737065635885), ('a', 0.1897425288444996)]
svd 
 [('long', 0.8733111261346901), ('above', 0.8259671977311955), ('around', 0.8030776291120685), ('sun', 0.7692439111243973), ('just', 0.7678481974778111), ('wide', 0.767257431992253), ('each', 0.7665960260861158), ('circle', 0.7647746702909336), ('length', 0.7601066921319761), ('almost', 0.7542351860536628)]


**Your answer should go here:**
Based on the natural language analogy 'light to heavy is short to long',

Assuming light -heavy ≈ short -long:

=> light -heavy +long ≈ short

=> light -(heavy -long) ≈ short

We'd expect 'short' to be among the top-1o similar words. The results in the cell above showed 'short' in the top-3 similar words in PPMI space, though the value is much lower than that of 'long' and 'light'.

Find 5 similar pairs of pairs of words and test them. Hint: Google for `word analogies examples`. You can also construct analogies that are less lexical but more grammatical, e.g. `see, saw, leave, ?` or analogies that are based on world knowledge as in the [Google analogy dataset](http://download.tensorflow.org/data/questions-words.txt) from [3]. Does the resulting vector similarity confirm your expectations? But remember you can only do this if the words are contained in our vector space with 10,000 dimensions. **[10 marks]**

In [147]:
# Your code should go here...
def analogize(pairA1, pairA2, pairB1 ):
    '''Given pairA and analogeous pairB (w1, w2=?), returns the most similar words to w2'''
    print(f'\n({pairA1}, {pairA2} <=> ({pairB1}, ?)')
    for name, space in {'raw':space_10k, 'ppmi':ppmispace_10k, 'svd':svdspace_10k}.items():
        a1 = normalize(space[pairA1])
        a2 = normalize(space[pairA2])
        b1 = normalize(space[pairB1])
#         b2 = normalize(space[pairB2])

        b2_sims = find_similar_to(b1-a1+a2, space)[:10]  # a1-a2=b1-b2  b2=b1-a1+a2
        print(name+': ', [duo[0] for duo in b2_sims])

        
# Semantic analogy: Vietnamese, Vietnam <=> Swedish, Sweden
analogize('vietnamese','swedish','vietnam')

# Syntactic analogy: go, went <=> sell, sold
analogize('go','went','sell')

# Knowledge analogy: Tokyo, Japan <=> Athens, Greece
analogize('athens','tokyo','greece')

# Hyponymy/hypernymy
analogize('apple','fruit','rice')

# Time: 
# Easter, spring <=> Christmas, winter
analogize('easter','spring','christmas')
# December, winter <=> July, summer
analogize('december','winter','july')


(vietnamese, swedish <=> (vietnam, ?)
raw:  ['iraq', 'kosovo', 'vietnam', 'amsterdam', 'germany', 'london', 'brazil', 'prague', 'paris', 'japan']
ppmi:  ['swedish', 'vietnam', 'danish', 'british', 'german', 'finnish', 'french', 'russian', 'dutch', 'norwegian']
svd:  ['zealand', 'czech', 'norwegian', 'austrian', 'swiss', 'danish', 'swedish', 'british', 'belgian', 'canada']

(go, went <=> (sell, ?)
raw:  ['went', 'returned', 'proceeded', 'sell', 'continued', 'decides', 'moved', 'chose', 'sought', 'manages']
ppmi:  ['sell', 'went', 'sold', 's', 'selling', 'buy', 'returned', 'began', 'purchase', 'produced']
svd:  ['bought', 'purchased', 'sold', 'acquired', 'announced', 'owned', 'donated', 'selling', 'shipped', 'traded']

(athens, tokyo <=> (greece, ?)
raw:  ['tokyo', 'italy', 'greece', 'germany', 'london', 'scandinavia', 'switzerland', 'india', 'singapore', 'argentina']
ppmi:  ['tokyo', 'greece', 'italy', 'turkey', 'bulgaria', 'hungary', 'croatia', 'germany', 'mexico', 'romania']
svd:  ['

Not every expected word appears in the top-10 similar words list. However, at least the list of words are in the same category as the expected word.

For example, despite 'Japan' not being among the similar words as expected for (athens, tokyo <=> (greece, ?), the words are all country names, which shows that at least the vectors aren't far away from the expected one.

In [150]:
'rat' in ktw_10k.tolist()

False

## 5. Semantic composition and phrase similarity **[20 marks]**

In this task, we are going to look at how different semantic composition models, introduced in [2] correlate with human judgements. The file with the dataset is `mitchell_lapata_acl08.txt` included with this notebook. Your task is to do the following:  

(i) process the dataset, extract pairs of `reference - landmark high` and `reference - landmark low`; you can use the code from the lecture as something to start with. Note that there are 2 landmarks for each reference: one landmark exhibits high similarity with the reference, while another one has low similarity with the reference. A single human participant could have evaluated both of these pairs. For more details, we refer you to the paper.  

(ii) build models of semantic phrase composition: in the lecture we introduced simple additive, simple multiplicative and combined models (details are in [2]). Your task is to take a single pair (a reference or a high similarity landmark or a low similarity landmark) and compute the composition of its vectors using each of these functions. Thus, you will have three compositional models that take a `noun - verb` pair and output a single vector, representing the meaning of this pair. As your semantic space, you can use pretrained spaces (standard space, ppmi or svd) introduced above. It is up to you which space you use, but for someone who runs your code, it should be pretty straightforward to switch between them.

(iii) calculate Spearman correlation between each model's predictions and human judgements; you should have something similar to the scores that are shown in the paper [2]:  

![title](./res.png)

The paper states that they calculated correlations between each individual participant's judgeements and each model's predictions.  

Let's say we have 3 models: simple additive (A), simple multiplicative (M), combined (C).  
From our task dataset, we also know that we have 20 participants.  
Now, for each participant in 20 participants we get all `verb - noun` pairs that these participated evaluated.  
For example:

In [151]:
participant_judgemenets_example = [
 'participant50 chatter child gabble 6 high',
 'participant50 chatter tooth click 2 high',
 'participant50 reel head whirl 5 high',
 'participant50 reel mind stagger 4 low',
 'participant50 reel industry stagger 5 high',
 'participant50 reel man whirl 3 low',
 'participant50 glow fire beam 7 low',
 'participant50 glow face burn 3 low',
 'participant50 glow cigar burn 5 high',
 'participant50 glow skin beam 7 high'
    
]

In [152]:
participant_judgemenets_example

['participant50 chatter child gabble 6 high',
 'participant50 chatter tooth click 2 high',
 'participant50 reel head whirl 5 high',
 'participant50 reel mind stagger 4 low',
 'participant50 reel industry stagger 5 high',
 'participant50 reel man whirl 3 low',
 'participant50 glow fire beam 7 low',
 'participant50 glow face burn 3 low',
 'participant50 glow cigar burn 5 high',
 'participant50 glow skin beam 7 high']

Let's look at the first pair that participant50 evaluated: reference `child chatter` and high-level similarity landmark (as the last word in the row indicates) `child gabble`. The human gave the similarity score of 6 (very similar). Thus, human similarity judgment = [6].  

Our A model's output:  
cosine(p1, p2) = 0.88, where p1 is the result of addition of word vectors in the reference phrase `child gabble`, and p2 is the result of addition of word vectors in the high-level similarity phrase `child chatter`.  

Therefore, we have human rating vector [6] and model A output [0.88]. Next is to compute correlation between these two vectors.

To get an overall score, simply average your correlation scores over all participants, since you are calculating correlation scores per participant.

Of course, your human rating vectors will be longer (e.g., [6, 7, 3, 4, 5]) where each element is a participant's judgement of a specific pair. Each of your models (A, B, C) will produce a single vector of cosine similarity between these same pairs (e.g., [0.89, 0.98, 0.23, 0.65, 0.55]). The goal is to compare each model's cosine similarity vectors with human rating vectors and identify the model which outputs the best result in terms of being the closest to the way human rate similarity between the phrases.

The minimum to do in this task: compute correlations for 3 models mentioned above and human rating for AT LEAST one participant. Elaborate on how different the resulting correlation scores are depending on the model's composition function (additive, multiplicative, combined). For examples on how to interpret the results, look at Section 5 Results of the original paper.

In [238]:
#your code should go here

with open('mitchell_lapata_acl08.txt') as f:
    lines = [l.rstrip('\n').split() for l in f]
    participants = {}
    for r in lines[1:]:
        participant, verb, noun, landmark, inputn, hilo = r
        refpair, landmarkpair = (noun,verb), (noun,landmark)
        pairs = (refpair, landmarkpair)
        inputn = int(inputn)
        
        # Init dictinary and keys:lists
        if participant not in participants:
            p = participants[participant] = {}
            p['high_pairs'],p['low_pairs'],p['high_inputs'],p['low_inputs'] = [],[],[],[]
        
        # Update dictinary
        p[f'{hilo}_pairs'].append(pairs)
        p[f'{hilo}_inputs'].append(inputn)
        
import pandas as pd            
pd.DataFrame.from_dict(participants).transpose()  # hilo-> researcher-judged ; input-> participant-scored


,high_pairs,low_pairs,high_inputs,low_inputs
participant20,"[((discussion, stray), (discussion, digress)),...","[((thought, stray), (thought, roam)), ((child,...","[6, 7, 5, 6, 4, 6, 7, 4, 2, 4, 6, 5, 2, 6, 7, ...","[7, 1, 2, 3, 4, 3, 2, 6, 4, 1, 4, 6, 1, 3, 4, ..."
participant21,"[((discussion, stray), (discussion, digress)),...","[((thought, stray), (thought, roam)), ((child,...","[7, 6, 4, 6, 4, 6, 5, 2, 1, 7, 6, 6, 7, 7, 7, ...","[5, 7, 1, 2, 3, 5, 3, 6, 2, 1, 4, 6, 1, 2, 1, ..."
participant22,"[((discussion, stray), (discussion, digress)),...","[((thought, stray), (thought, roam)), ((child,...","[4, 3, 5, 6, 6, 6, 6, 6, 6, 3, 5, 5, 6, 7, 2, ...","[5, 2, 2, 5, 1, 3, 2, 2, 4, 3, 6, 1, 2, 6, 3, ..."
participant23,"[((discussion, stray), (discussion, digress)),...","[((thought, stray), (thought, roam)), ((child,...","[4, 5, 6, 5, 7, 5, 2, 2, 3, 4, 2, 5, 2, 5, 4, ...","[4, 2, 2, 2, 3, 1, 5, 5, 3, 4, 1, 3, 3, 6, 2, ..."
participant24,"[((discussion, stray), (discussion, digress)),...","[((thought, stray), (thought, roam)), ((child,...","[6, 6, 5, 6, 5, 6, 1, 5, 4, 4, 4, 6, 5, 6, 3, ...","[6, 3, 5, 2, 4, 1, 1, 1, 1, 4, 3, 4, 2, 2, 6, ..."
participant25,"[((discussion, stray), (discussion, digress)),...","[((thought, stray), (thought, roam)), ((child,...","[3, 6, 2, 5, 3, 5, 3, 3, 2, 6, 4, 6, 5, 5, 6, ...","[3, 4, 2, 5, 4, 2, 4, 4, 1, 3, 2, 6, 1, 5, 3, ..."
participant26,"[((discussion, stray), (discussion, digress)),...","[((thought, stray), (thought, roam)), ((child,...","[4, 7, 4, 6, 4, 5, 1, 6, 6, 4, 3, 5, 5, 5, 5, ...","[7, 3, 2, 2, 1, 3, 1, 1, 2, 4, 2, 4, 5, 4, 7, ..."
participant27,"[((discussion, stray), (discussion, digress)),...","[((thought, stray), (thought, roam)), ((child,...","[7, 4, 6, 7, 5, 4, 7, 4, 2, 5, 7, 7, 7, 4, 6, ...","[5, 2, 3, 4, 5, 3, 2, 6, 3, 2, 2, 3, 2, 2, 3, ..."
participant28,"[((discussion, stray), (discussion, digress)),...","[((thought, stray), (thought, roam)), ((child,...","[7, 6, 7, 7, 5, 4, 6, 6, 4, 6, 6, 7, 7, 5, 6, ...","[6, 1, 2, 3, 4, 2, 2, 1, 2, 1, 2, 1, 1, 2, 3, ..."
participant29,"[((discussion, stray), (discussion, digress)),...","[((thought, stray), (thought, roam)), ((child,...","[6, 4, 4, 5, 5, 5, 5, 2, 3, 3, 5, 6, 2, 3, 3, ...","[5, 2, 4, 3, 1, 5, 4, 2, 5, 3, 5, 2, 2, 3, 4, ..."


**Any comments/thoughts should go here:**

In [275]:
space = svdspace_10k

import numpy as np
def add_vectors( phrase_tuple, space ):
    
    default_vec = np.zeros((len(next(iter(space.values())))), dtype=int) # array of zeros, length=numdim
    wordvec = lambda word : space.get(word, default_vec)
    w1,w2 = phrase_tuple
    return wordvec(w1)+wordvec(w2)

add_vectors( ('child','stray'), space)

array([-1.68596795e+01,  1.12632093e+00,  5.52293887e+00, -1.91521912e-01,
        6.26589740e+00,  6.91026887e-01,  1.86197037e+00,  1.57630156e-02,
       -4.70677930e-01, -1.25399636e+00, -2.99762462e+00, -3.94471027e+00,
        2.08396734e+00,  3.60989887e-01, -1.77174486e-01, -1.72300995e-01,
       -1.96353917e+00,  1.58277370e+00, -4.66993473e+00,  6.98337037e-01,
        3.48305837e+00, -7.20262415e-01,  3.58028058e+00, -4.88544487e-02,
       -2.67460010e+00,  4.07908401e-02,  3.01909002e+00,  3.68060928e+00,
        1.36321028e+00,  9.88181573e-01, -4.50114629e+00,  3.24485175e+00,
       -5.86159167e-01, -1.77767595e+00,  9.31526874e-02,  6.97726003e-01,
       -2.19314277e+00,  1.61373383e+00, -2.89539438e-01, -8.96391996e-01,
       -9.05775123e-01, -3.71644535e-01,  1.77052864e+00, -4.48000291e-01,
        2.31402803e+00, -5.95450446e-01, -4.06998102e-01,  2.87939772e+00,
        2.52377284e+00, -2.34651227e+00])

In [280]:
import math


def cos_sim(vec1, vec2):
 
    vlen = lambda v:math.sqrt(numpy.sum(numpy.square(v)))
    vlen1, vlen2 = vlen(vec1), vlen(vec2)

    if vlen1 == 0.0 or vlen2 == 0.0:
        # one of the vectors is empty. make the cosine zero.
        return 0.0

    else:
        # we could also simply do:
        # dotproduct = numpy.dot(vec1, vec2)
        dotproduct = numpy.sum(vec1 * vec2)

        return dotproduct / (vlen1 * vlen2)

In [301]:
the_participant = 'participant20'
p = participants[the_participant]

from statistics import mean
hi_vec = p['high_inputs']
lo_vec = p['low_inputs']

print(mean(hi_vec), mean(lo_vec)) # scale from 1~7

hi_add_vec = [ cos_sim( add_vectors(pair1, space), add_vectors(pair2, space) ) for pair1, pair2 in p['high_pairs'] ]
lo_add_vec = [ cos_sim( add_vectors(pair1, space), add_vectors(pair2, space) ) for pair1, pair2 in p['low_pairs'] ]
print(mean(hi_add_vec), mean(lo_add_vec))

5.25 3.857142857142857
0.88355212196572 0.8276639436533901


In [310]:
concat_tolist = lambda arr1, arr2 : np.concatenate((arr1,arr2)).tolist()   # Concatenates two arrays and turn into a list
human = concat_tolist(hi_vec, lo_vec)
add_vecs = concat_tolist(hi_add_vec,lo_add_vec)
spearman_compare( human, add_vecs)



    rho     = -0.0035
    p-value = 0.9788



# Literature

  - [1] C. Silberer and M. Lapata. Learning grounded meaning representations with autoencoders. In Proceedings of the 52nd Annual Meeting of the Association for Computational Linguistics, pages 721–732, Baltimore, Maryland, USA, June 23–25 2014 2014. Association for Computational Linguistics.  

  - [2] Mitchell, J., & Lapata, M. (2008). Vector-based Models of Semantic Composition. In Proceedings of ACL-08: HLT (pp. 236–244). Association for Computational Linguistics.
  
  - [3] T. Mikolov, I. Sutskever, K. Chen, G. S. Corrado, and J. Dean. Distributed representations of words and phrases and their compositionality. In Advances in neural information processing systems, pages 3111–3119, 2013.

## Marks

This assignment has a total of 60 marks.